# 101 Asynchronous programming with Python
## Module 1

### Agenda:

* Key definitions
* What is "async" and "asynchronous".
* Coroutine objects, `async` and `await` keywords.

# Key definitions

## Concurrency and parallelism

**Concurrent computing**

<div align="center"><img src="../images/whooper%20swans%20on%20snow.jpg?raw=1" alt="whooper swans on snow" width="200"/></div>

is a form of computing in which several computations are executed concurrently—during overlapping time periods—instead of sequentially, with one completing before the next starts.

<div align="right">
    – <a href="https://en.wikipedia.org/wiki/Concurrent_computing">Wikipedia / Concurrent computing </a>
</div>

---



**Parallelism (parallel computing)**

<div align="center"><img src="../images/three%20seagulls.jpg?raw=1" alt="three seagulls" width="200"/></div>

is a simultaneous execution of different parts of a program on multiple processors.

<div align="right">
    – <a href="https://en.wikipedia.org/wiki/Parallel_computing">Wikipedia / Parallel computing </a>
</div>

👉 *You may have parallelism without concurrency, and concurrency without parallelism.* 👈

---

## Multiprocessing, multithreading, multitasking
**(in Python)**

**Multiprocessing**

<div align="center"><img src="../images/seagulls%20in%20a%20row.jpg" alt="seagulls in a row" width="200"/></div>

The load is distributed between multiple processes that have an
ability to run in parallel on different CPUs.  It's good for heavy
computation tasks, and allows to achieve a real parallelism.

Still, spawning processes is expensive, and they may be hard to manage
and communicate.  *If things go wrong, you may end up with
orphan processes or an army of zombies.*

---
**Multithreading**
<div align="center"><img src="../images/geese.jpg" alt="geese" width="200"/></div>

Threads are "execution units" of a process, managed by OS.  They are
more easy to create.  Since threads share resources, you can convert
your single-thread code to a multithreaded one with just a bit of
changes.

On the other hand, sharing resources and OS-managed-context-switching
means that you need to pay attention to tread safety.  It also
add complications to signal handling (like raising KeyboardInterrupt).

Threads are expensive, still not so expensive as processes.
In Python they allow to execute the code concurrently,
but not in parallel.

---

**Multitasking**
<div align="center"><img src="../images/four%20rock%20doves%20on%20gray%20floor.jpg" alt="four rock doves on gray floor" width="200"/></div>

Allows to execute concurrent tasks that share common computation
resources, such as CPU and RAM.

With **cooperative multitasking**, the process (the program) decides
itself when to interrupt its execution and yield control to
other processes.

Cooperative multitasking allows simpler implementations, since there is
no implicit context switching and no thread safety is needed.
It allows to have a high number of tasks executed concurrently.

Still, creating applications with cooperative multitasking require
care to make sure each task fairly use the computation resources
and timely yields control.  Also a high number of tasks still
can affect the application performance.

---

### Questions

**1. Can you see, when it is better to use multiprocessing in your code? Multithreading? Multitasking?**

**2. This is how one could create an infinitely-sleeping program:**

```python
from time import sleep

while True:
    sleep(1)
```

**Why `sleep(1)` is needed here?**

# What is "async" and "asynchronous"

**Asynchronous code**

<div align="center"><img src="../images/finches%20and%20a%20feeder.jpg" alt="finches and a feeder" width="200"/></div>

is code that deals with events that occur independently of the main program flow.

One of the ways to create such code for cooperative multitasking is to use **coroutines** – objects that provide an abstraction of an ongoing event.

Coroutines provide an interface to suspend and resume their execution.

### Let's write our own asynchronous code

First, let's simulate some external events, like a user pressing one of the arrow keys on a keyboard.  The code below is rather complex, but here what it does.

For every `period` (1 second by default), a `UserInput` instance sets a value of a key, pressed by a simulated user.  By default there is only a 30% probability that the value is to be one of the arrow keys, and 70% probability that the value is None (no event exists) for a given `period`.

In [15]:
from dataclasses import dataclass
from random import choice, random
from time import monotonic, sleep
from typing import Iterable, Optional
from warnings import warn


@dataclass
class UserInput:
    """User input representation."""
    period: float = 1.0
    choices: Iterable[str] = "⟹⟸⇑⇓"
    pressed_probability: float = 0.3

    def __post_init__(self):
        self._last_event_time = monotonic()
        self._last_event = None

    def get_key(self) -> Optional[str]:
        """Decide if a user pressed some key, and return its symbol."""
        current_time = monotonic()
        time_since_last_event =  current_time - self._last_event_time
        if time_since_last_event < self.period:
            return self._last_event
        
        if time_since_last_event / 2 > self.period:
            warn(
                f"Getting the pressed key is delayed by { time_since_last_event - self.period } seconds, "
                + "previous events may be lost."
            )

        self._last_event_time = current_time - (time_since_last_event % self.period)
        self._last_event = choice(self.choices) if self.pressed_probability > random() else None

        return self._last_event

Here is an example on how it works:

In [16]:
from time import monotonic, sleep


user_input = UserInput(1)


for _ in range(10):
    print("Key pressed:", user_input.get_key())
    sleep(1)

Key pressed: None
Key pressed: ⟹
Key pressed: ⇑
Key pressed: None
Key pressed: None
Key pressed: ⇓
Key pressed: None
Key pressed: ⟸
Key pressed: ⇑
Key pressed: None


---
If you're familar with [generators](https://docs.python.org/3/glossary.html#term-generator), here is how you could use one to implement a coroutine that handles a user input:

In [17]:
def handle_character_move(user_input: UserInput) -> Iterable:
    """Get the user input and do something."""
    movements = dict(zip("⟹⟸⇑⇓", ["right", "left", "up", "down"]))
    
    while True:
        key_pressed = user_input.get_key()    
        yield movements.get(key_pressed, "stay")


user_input = UserInput(1)


for _, action in zip(range(10), handle_character_move(user_input)):
    print("MAIN FLOW: Key pressed:", user_input.get_key())
    print("COROUTINE:", action)
    sleep(1)

MAIN FLOW: Key pressed: None
COROUTINE: stay
MAIN FLOW: Key pressed: None
COROUTINE: stay
MAIN FLOW: Key pressed: ⟸
COROUTINE: left
MAIN FLOW: Key pressed: None
COROUTINE: stay
MAIN FLOW: Key pressed: ⟸
COROUTINE: left
MAIN FLOW: Key pressed: None
COROUTINE: stay
MAIN FLOW: Key pressed: None
COROUTINE: stay
MAIN FLOW: Key pressed: None
COROUTINE: stay
MAIN FLOW: Key pressed: None
COROUTINE: stay
MAIN FLOW: Key pressed: None
COROUTINE: stay


---
Now, a little bit of refactoring, to call `user_input.get_key()` just once.  Did you know that you could [send values](https://docs.python.org/3/reference/expressions.html#generator.send) to the generator-coroutine?  Also, to save resources, let's wake it up only when there is some user input.

In [18]:
def handle_character_move() -> Iterable:
    """Get the user input and do something."""
    movements = dict(zip("⟹⟸⇑⇓", ["right", "left", "up", "down"]))
    key_pressed = None
    
    while True:
        action = movements.get(key_pressed, "stay")
        key_pressed = (yield action)


user_input = UserInput(1)

# Initialize a generator
character_move_coroutine = handle_character_move()

# Let it run till the first `yield`.
# `character_move_coroutine.send(None)` would also work:
next(character_move_coroutine)

try:
    for _ in range(10):
        key = user_input.get_key()
        print("MAIN FLOW: Key pressed:", key)
        if key is not None:
            print("COROUTINE:", character_move_coroutine.send(key))
        sleep(1)
finally:
    character_move_coroutine.close()

MAIN FLOW: Key pressed: None
MAIN FLOW: Key pressed: None
MAIN FLOW: Key pressed: None
MAIN FLOW: Key pressed: None
MAIN FLOW: Key pressed: None
MAIN FLOW: Key pressed: None
MAIN FLOW: Key pressed: None
MAIN FLOW: Key pressed: ⇓
COROUTINE: down
MAIN FLOW: Key pressed: ⇑
COROUTINE: up
MAIN FLOW: Key pressed: None


### Questions

**1. The example above shouws you how an event loop and a coroutine works.  Do you see a problem with this event loop implementation?**

**1.1. What if you make events apper faster? Try to increase the speed of events occurrence by changing `user_input = UserInput(1)` to `user_input = UserInput(0.1)`**

**2. Compare the example above to some of the other applications:**

* [pygame](https://www.pygame.org/docs/tut/PygameIntro.html#taste);
* [tkinter](https://docs.python.org/3/library/tkinter.html#a-simple-hello-world-program);
* [curio](https://curio.readthedocs.io/en/latest/tutorial.html#getting-started).

Do you see where the event loop is started there?

**3. Generators have `send` and `trow` methods.  What are they needed for?**

👉 *Coroutines are not the only way to create asynchronous code, but the most readable one.  And `async` is the syntax sugar for a coroutine definition* 👈

# Coroutine objects

---
Let's create a coroutine and examine it.

In [19]:
async def coro():
    print("I am a coroutine!")

In [20]:
coroutine = coro()
coroutine

<coroutine object coro at 0x7fa59f600840>

In [21]:
dir(coroutine)

['__await__',
 '__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'cr_await',
 'cr_code',
 'cr_frame',
 'cr_origin',
 'cr_running',
 'send',
 'throw']

In [22]:
coroutine.send(None)

I am a coroutine!


StopIteration: 

👉 *Like generators, coroutines have `send` and `throw` methods, and raise `StopIteration` when exhousted.* 👈

But, unlike generators, you can not iterate a coroutine till the end, since there are no `__iter__` and `__next__` methods.  Instead, there is `__await__` method, so you can `await` for it.

In [23]:
async def no_op():
    return


async def async_coroutine():
    for i in range(5):
        print("async_coroutine iteration", i)
        await no_op()


def sync_coroutine():
    for i in range(5):
        print("sync_coroutine iteration", i)
        yield
    

In [24]:
for _ in sync_coroutine():
    pass


print("=" * 17)


for _ in async_coroutine().__await__():
    pass


sync_coroutine iteration 0
sync_coroutine iteration 1
sync_coroutine iteration 2
sync_coroutine iteration 3
sync_coroutine iteration 4
async_coroutine iteration 0
async_coroutine iteration 1
async_coroutine iteration 2
async_coroutine iteration 3
async_coroutine iteration 4


👉 *Using `async` and `await` does not automatically make your code asynchronous.* 👈

---
Coroutines and generators look similar, also there is one key difference: `__await__` lets processing the whole coroutine on one go.  The behavior here is more close to `yield from` comparing to `yield`

In [25]:
from time import sleep


for i, _ in enumerate(sync_coroutine()):
    print("outer sync_coroutine iteration:", i)
    sleep(1)


print("=" * 17)


for i, _ in enumerate(async_coroutine().__await__()):
    print("outer async_coroutine iteration:", i)
    sleep(1)


sync_coroutine iteration 0
outer sync_coroutine iteration: 0
sync_coroutine iteration 1
outer sync_coroutine iteration: 1
sync_coroutine iteration 2
outer sync_coroutine iteration: 2
sync_coroutine iteration 3
outer sync_coroutine iteration: 3
sync_coroutine iteration 4
outer sync_coroutine iteration: 4
async_coroutine iteration 0
async_coroutine iteration 1
async_coroutine iteration 2
async_coroutine iteration 3
async_coroutine iteration 4


Coroutine will yield only if some underlying code have to yield.

In [26]:
class Awaitable:
    def __await__(self):
        for i in range(3):
            print("index from awaitable:", i)
            yield i
        return "Ok"


async def async_coroutine():
    awaitable = Awaitable()
    for i in range(5):
        print("coroutine iteration", i)
        result = await awaitable
        print("Awaitable says:", result)
        print("=" * 21)
        


for i, j in enumerate(async_coroutine().__await__()):
    print("outer iteration:", i)
    print("coroutine iteration result:", j)
    print("-" * 23)
    sleep(1)


coroutine iteration 0
index from awaitable: 0
outer iteration: 0
coroutine iteration result: 0
-----------------------
index from awaitable: 1
outer iteration: 1
coroutine iteration result: 1
-----------------------
index from awaitable: 2
outer iteration: 2
coroutine iteration result: 2
-----------------------
Awaitable says: Ok
coroutine iteration 1
index from awaitable: 0
outer iteration: 3
coroutine iteration result: 0
-----------------------
index from awaitable: 1
outer iteration: 4
coroutine iteration result: 1
-----------------------
index from awaitable: 2
outer iteration: 5
coroutine iteration result: 2
-----------------------
Awaitable says: Ok
coroutine iteration 2
index from awaitable: 0
outer iteration: 6
coroutine iteration result: 0
-----------------------
index from awaitable: 1
outer iteration: 7
coroutine iteration result: 1
-----------------------
index from awaitable: 2
outer iteration: 8
coroutine iteration result: 2
-----------------------
Awaitable says: Ok
coro

---
Take a look at `coroutine iteration result` printed out of the main loop.  You can see that the loop and `Awaitable` keep chatting one to another, while `async_coroutine` waits for `Awaitable` to return some final value.

That's how a typical `async`/`await` application looks like:

<div align="center"><img src="../images/birds%20on%20a%20wire.jpg" alt="birds on a wire" width="200"/></div>

The main loop at one end.
Awaitables that know how to communicate to the loop – at the other end.

And your code pipes one end to another via `async` and `await`.

![piping async code](../images/piping-async-code.png)

Examples of such "special awaitables" are `asyncio.sleep`, `curio.sleep` and `trio.sleep`.

---
Let's now reimplement our example of a fake movement for a fake user input.

In [27]:
# Your code is here:

async def handle_character_move() -> Iterable:
    """Get the user input and do something."""
    movements = dict(zip("⟹⟸⇑⇓", ["right", "left", "up", "down"]))
    
    for _ in range(5):
        key_pressed = await get_key_pressed()
        print("ACTION:", movements.get(key_pressed, "stay"))


# The async framework code is below:

class InputProvider:
    """Awaitable that redirects input to the awaiting coroutine."""
    def __await__(self):
        output = None
        while output is None:
            print(".", end="")
            output = yield
        print()
        return output


async def get_key_pressed():
    """Provide pressed key character."""
    return await InputProvider()


# Here the event loop is inside:

def run(coroutine):
    """Run a coroutine."""
    from time import sleep

    user_input = UserInput(1)

    try:
        # Run the coroutine until first downstream `yield`
        coroutine.send(None)

        while True:
            coroutine.send(user_input.get_key())
            sleep(1)

    except StopIteration:
        print("Bye!")


# Plug it in!
run(handle_character_move())

......
ACTION: left
....
ACTION: left
.
ACTION: down
...
ACTION: up
..
ACTION: down
Bye!


# Conclusions

1. For simple cases of concurency consider using multithreading.  If you need heavy computations or true parallelism, use multiprocessing approach, or even a distributed application.
2. Multitasking and coroutines are good for I/O operations.  Also *multiprocessing applications* can be written in asynchronous style.
3. Python does great job hiding implementation details behind `async` and `await` keywords, making them look similar to regular synchronous code.
4. Python coroutines do not need `asyncio` to be executed.  `asyncio` is just one of the asynchronous frameworks that are able to run your asyncronous code.

<span style="font-size: x-large">Add your code below:</span>